In [3]:
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dropout, Dense

def predict_next_day_values(data_path):
    # Load data from JSON file
    with open(data_path) as f:
        data = json.load(f)

    # Extract features and target
    X = np.column_stack((data['o'], data['h'], data['l'], data['c'], data['v']))
    y = np.array(data['c'])  # Close price

    # Normalize data
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(y.reshape(-1, 1))

    # Reshape data for LSTM input (samples, timesteps, features)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)

    # Build the model
    model = Sequential()
    model.add(Bidirectional(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1))))
    model.add(Dropout(0.2))
    model.add(Bidirectional(GRU(units=50)))
    model.add(Dropout(0.2))
    model.add(Dense(32, kernel_initializer="uniform", activation="relu"))
    model.add(Dense(1, kernel_initializer="uniform", activation="linear"))
    model.compile(optimizer="adam", loss="mean_squared_error")

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=1024, verbose=1)

    # Predict tomorrow's closing price
    last_data_point = X_scaled[-1].reshape(1, X_scaled.shape[1], 1)
    predicted_price_scaled = model.predict(last_data_point)
    predicted_price = scaler.inverse_transform(predicted_price_scaled)

    return predicted_price

# Example usage:
predicted_price = predict_next_day_values('data.json')
print("Predicted closing price for tomorrow:", predicted_price)


Epoch 1/50
4/4 [==============================] - 3s 77ms/step - loss: 0.2008
Epoch 2/50
4/4 [==============================] - 0s 27ms/step - loss: 0.1821
Epoch 3/50
4/4 [==============================] - 0s 27ms/step - loss: 0.1530
Epoch 4/50
4/4 [==============================] - 0s 26ms/step - loss: 0.1047
Epoch 5/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0412
Epoch 6/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0172
Epoch 7/50
4/4 [==============================] - 0s 27ms/step - loss: 0.0283
Epoch 8/50
4/4 [==============================] - 0s 33ms/step - loss: 0.0101
Epoch 9/50
4/4 [==============================] - 0s 26ms/step - loss: 0.0126
Epoch 10/50
4/4 [==============================] - 0s 27ms/step - loss: 0.0129
Epoch 11/50
4/4 [==============================] - 0s 28ms/step - loss: 0.0075
Epoch 12/50
4/4 [==============================] - 0s 28ms/step - loss: 0.0057
Epoch 13/50
4/4 [==============================] - 0s 26ms/st

In [39]:
class EpochPrinter(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1} completed.")


class StockPredictor:
    def __init__(self, data_path):
        self.data_path = data_path
        self.models = {}
        self.history = {}
        self.scalers = {}

    def preprocess_data(self):
        with open(self.data_path) as f:
            data = json.load(f)

        X = np.column_stack((data["o"], data["h"], data["l"], data["c"], data["v"]))

        scalers = {}
        for key in ["o", "h", "l", "c", "v"]:
            scalers[key] = MinMaxScaler()
            if key == "o":
                y_scaled = scalers[key].fit_transform(
                    np.array(data[key]).reshape(-1, 1)
                )
            else:
                scalers[key].fit(np.array(data[key]).reshape(-1, 1))
        self.scalers = scalers
        X_scaled = MinMaxScaler().fit_transform(X)
        X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

        self.X_train, self.X_test, self.y_train_dict, self.y_test_dict = {}, {}, {}, {}
        for key in ["o", "h", "l", "c", "v"]:
            (
                self.X_train[key],
                self.X_test[key],
                self.y_train_dict[key],
                self.y_test_dict[key],
            ) = train_test_split(
                X_scaled,
                scalers[key].transform(np.array(data[key]).reshape(-1, 1)),
                test_size=0.3,
                random_state=42,
            )

    def build_models(self):
        for key in ["o", "h", "l", "c", "v"]:
            model = Sequential()
            model.add(
                Bidirectional(
                    GRU(
                        units=50,
                        return_sequences=True,
                        input_shape=(self.X_train[key].shape[1], 1),
                    )
                )
            )
            model.add(Dropout(0.2))
            model.add(Bidirectional(GRU(units=50)))
            model.add(Dropout(0.2))
            model.add(Dense(32, kernel_initializer="uniform", activation="relu"))
            model.add(Dense(1, kernel_initializer="uniform", activation="linear"))
            model.compile(optimizer="adam", loss="mean_squared_error")
            self.models[key] = model

    def inverse_transform_predictions(self, predicted_values):
        inv_predicted_values = {}
        for key in ["o", "h", "l", "c", "v"]:
            inv_predicted_values[key] = self.scalers[key].inverse_transform(
                predicted_values[key]
            )
        return inv_predicted_values

    def train_models(self, epochs=50, batch_size=32):
        callbacks = [EpochPrinter()]  # Callback để in ra số lượng epoch
        for key in ["o", "h", "l", "c", "v"]:
            self.history[key] = self.models[key].fit(
                self.X_train[key],
                self.y_train_dict[key],
                epochs=epochs,
                batch_size=batch_size,
                verbose=0,
                callbacks=callbacks,
            )

    def predict_next_day_values(self):
        last_data_point = self.X_test["o"][-1].reshape(1, self.X_test["o"].shape[1], 1)
        predicted_values = {}
        for key in ["o", "h", "l", "c", "v"]:
            predicted_values[key] = self.models[key].predict(last_data_point)
        return self.inverse_transform_predictions(predicted_values)

    def save_models(self, save_dir="models/"):
        for key, model in self.models.items():
            model.save(f"{save_dir}{key}_model.h5")

    def load_models(self, load_dir="models/"):
        for key in ["o", "h", "l", "c", "v"]:
            self.models[key] = load_model(f"{load_dir}{key}_model.h5")

In [40]:
predictor = StockPredictor("data.json")


In [41]:
predictor.preprocess_data()


In [42]:
predictor.build_models()

In [43]:
predictor.load_models()


In [44]:
predicted_values = predictor.predict_next_day_values()

1/1 [==============================] - 1s 928ms/step


In [ ]:
# Example usage:
predictor.train_models(epochs=100)
predictor.save_models()
predicted_values_loaded = predictor.predict_next_day_values()

# In ra số lượng epoch đã hoàn thành
for key in ["o", "h", "l", "c", "v"]:
    print(f"Epochs completed for {key}:", len(predictor.history[key].history["loss"]))



In [45]:
print("Predicted Open price for tomorrow:", predicted_values["o"])
print("Predicted High price for tomorrow:", predicted_values["h"])
print("Predicted Low price for tomorrow:", predicted_values["l"])
print("Predicted Close price for tomorrow:", predicted_values["c"])
print("Predicted Volume for tomorrow:", predicted_values["v"])

Predicted Open price for tomorrow: [[949.3951]]
Predicted High price for tomorrow: [[948.6303]]
Predicted Low price for tomorrow: [[929.8655]]
Predicted Close price for tomorrow: [[945.2864]]
Predicted Volume for tomorrow: [[1.5815574e+08]]
